In [1]:
from pathlib import Path
from typing import List
import gzip
import numpy as np
import pandas as pd

import torch.utils.data as Data 
import torch
import matplotlib.pyplot as plt
from torch import nn ,optim


In [2]:
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("GPU获取")

GPU获取


In [4]:
data_participants_dir='E:/Data/participants1.tsv'#获取全部实验者的数据
data_participants_tsv=pd.read_csv(data_participants_dir,sep='\t')
data_NoNaN=data_participants_tsv.dropna(axis=0,how='any')#去除NAN无效数据
data_RA=data_NoNaN.raven_score.head(100)

In [5]:
data_NoNaN

,participant_id,age,sex,BMI,handedness,education_category,religious_now,raven_score,NEO_N,NEO_E,NEO_O,NEO_A,NEO_C
0,sub-0001,26.25,F,24.0,right,applied,no,29.0,35,44,37,39,48
1,sub-0002,21.00,F,23.0,right,academic,no,27.0,27,53,40,48,34
2,sub-0003,23.00,M,22.0,right,academic,no,22.0,35,40,43,37,38
3,sub-0004,23.50,F,22.0,right,applied,no,21.0,29,47,41,48,41
4,sub-0005,21.75,M,19.0,ambidextrous,applied,no,28.0,26,35,40,38,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,sub-0212,20.50,F,29.0,ambidextrous,applied,no,19.0,24,44,29,47,41
212,sub-0213,19.75,M,26.0,right,applied,no,18.0,20,50,40,46,39
213,sub-0214,19.50,M,19.0,right,applied,no,19.0,24,50,37,46,56
214,sub-0215,18.25,F,20.0,right,applied,yes,24.0,33,42,42,41,48


In [ ]:
spatial_map_trian = np.load('/input/spatial_map_2train.npy')#获取样本数据
len(spatial_map_trian)

In [ ]:
def create_data_iter(data,target,bs):
    train_target=torch.tensor(target,dtype=torch.float)
    spatial_map_trian=torch.tensor(data, dtype=torch.float)#转化成tensor类别
    batch_size=bs
    dataset = Data.TensorDataset(spatial_map_trian, train_target)
    data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
    return data_iter
    

In [ ]:
batch_size=20
data_iter=create_data_iter(spatial_map_trian,data_RA,batch_size)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv=nn.Sequential(
            nn.Conv3d(1,4,12,4),#240*240*240->58*58*58
            nn.BatchNorm3d(4),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#58*58*58->29*29*29
            nn.Conv3d(4,8,3,1,1),#29*29*29->29*29*29
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#29*29*29->14*14*14
            nn.Conv3d(8,16,3,1,1),#14*14*14->14*14*14
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#14*14*14->7*7*7
            nn.Conv3d(16,32,3,1,1),#7*7*7->7*7*7
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(2,2)#7*7*7->3*3*3
            
        )
        self.fc=nn.Sequential(
            nn.Linear(32*3*3*3,512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64,1)
         )
    def forward (self,img):
        padding=nn.ConstantPad3d((10,10,0,0,0,0),0.0)
        img=padding(img)
        batch_size, x, y, z = img.shape
        img = img.reshape(batch_size, 1, x, y, z)
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

net=AlexNet()

In [ ]:
net=net.to(device)
loss=nn.MSELoss()
result=[]
lr=0.001
n=100/batch_size
num_epoches=300
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
for epoch in range(num_epoches):
    train_l_sum =0.0
    for X,y in data_iter:
        X=X.to(device)
        y=y.to(device)
        y=y.reshape(len(y),-1)
        y_hat=net(X)
        l=loss(y_hat,y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_l_sum+=l.cpu().item()
        
    if(epoch%10==0):
        result.append(train_l_sum / n)
        print('epoch %d, loss %.4f'%(epoch + 1, train_l_sum / n))

In [2]:
def evaluate_acc(data,net,bs):
    test_l_sum,n=0.0,100/bs
    for X,y in data:
        
        X=X.to(device)
        y=y.to(device)
        y=y.reshape(len(y),-1)
        y_hat=net(X)
        l=loss(y_hat,y)
        test_l_sum+=l.cpu().item()
    return test_l_sum / n

In [4]:
data_RA2=data_NoNaN.raven_score.tail(100)
data_RA2_array=np.array(data_RA2)

In [ ]:
spatial_map_test = np.load('/input/spatial_map_2vaild.npy')

In [ ]:
data_iter_test=create_data_iter(spatial_map_test,data_RA2_array,batch_size)

In [ ]:
result_train=[]
result_test=[]
for i in range(10):
    result_train.append(evaluate_acc(data_iter,net,batch_size))
    result_test.append(evaluate_acc(data_iter_test,net,batch_size))

In [ ]:
data_participants_dir='/input/participants2.tsv'#获取全部实验者的数据
data_participants_tsv2=pd.read_csv(data_participants_dir,sep='\t')
data_NoNaN2=data_participants_tsv2.dropna(axis=0,how='any')#去除NAN无效数据
data_RA3=data_NoNaN2.raven_score.tail(100)

In [ ]:
spatial_map_vaild= np.load('/input/spatial_map_test.npy')#获取样本数据
len(spatial_map_vaild)

In [ ]:
del data_iter

In [ ]:
data_RA3=data_NoNaN2.raven_score.tail(100)
data_RA3_array=np.array(data_RA3)
data_iter_vaild=create_data_iter(spatial_map_vaild,data_RA3_array,batch_size)

In [ ]:
result_vaild=[]
for i in range(10):
    
    result_vaild.append(evaluate_acc(data_iter_vaild,net,batch_size))

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
name_list=['1','2','3','4','5','6','7','8','9','10']
width=0.4
x =list(range(len(name_list)))  
plt.plot(x, result_train, marker='o', mec='r', mfc='w',label='PIOP2train')
plt.plot(x, result_test, marker='*', ms=10,label='PI0P2test')
plt.plot(x, result_vaild, marker='^', ms=10,label='PI0P1test')
plt.legend()  # 让图例生效